In [ ]:
from itertools import product
import numpy as np
from sklearn.linear_model import LinearRegression

In [ ]:
# Generate data
np.random.seed(seed=5)
p = 2
n = 1000
total_subsets = 2 ** p
total_individuals = 4
ones = np.full(shape=(n, 1), fill_value=1)
X = np.random.normal(loc=0, scale=1, size=(n, p))
X_intercept = np.concatenate((ones, X), axis=1)
beta = np.reshape(np.arange(1, p+2), newshape=(p+1, 1))
y = np.matmul(X_intercept, beta) + np.random.normal(loc=0, scale=1, size=(n, 1))
beta_hat = np.matmul(np.matmul(np.linalg.inv(np.matmul(X_intercept.T, X_intercept)), X_intercept.T), y)
print('beta:\n', beta)
print('beta_hat:\n', beta_hat)

In [ ]:
# Point to be described
x_new = np.array([[0, 2]])
x_new_intercept = np.array([[1, 0, 0]])
print(x_new_intercept.shape)
print(beta.shape)

In [ ]:
# Black box model
linear_model = LinearRegression(fit_intercept=True)
linear_model = linear_model.fit(X, y)

In [ ]:
print('real betas:\n', beta)
print('intercept:', linear_model.intercept_)
print('coef:', linear_model.coef_)
print('real vale for x_new:', np.dot(x_new_intercept, beta))
print('predicted value for x_new:', linear_model.predict(x_new))

In [ ]:
# Obtain matrix Z
def obtain_Z(p, total_individuals):
    all_subsets = product([0, 1], repeat=p)
    idx_individuals = np.random.choice(a=total_subsets, size=total_individuals, replace=False)
    list_individuals = np.random.permutation([x for i, x in enumerate(all_subsets) if i in idx_individuals])
    Z = np.array(list_individuals, dtype=int)
    return Z
Z = obtain_Z(p, total_individuals)
Z[:3, ]

In [ ]:
# Obtain matrix H
# TO DO: run this in parallel
def obtain_H(Z, data, x_new, verbose=False):
    n_rows_data = data.shape[0]
    p = Z.shape[1]
    idx_columns = range(p)
    ones_vector = np.full(shape=(n_rows_data, 1), fill_value=1)
    total_ones_Z = np.sum(Z, axis=1)
    idx_non_zeros_Z = np.where(total_ones_Z>0)
    idx_non_zeros_Z = idx_non_zeros_Z[0]
    Z_filtered = Z[idx_non_zeros_Z, :]
    n_rows_Z_filtered = Z_filtered.shape[0]
    H = np.empty(shape=(n_rows_Z_filtered, p))

    if verbose:
        print('Z_filtered:\n', Z_filtered)
        print('data:\n', data[:3, ])
        print('initial H:\n', H)
        print('x_new:\n', x_new)
        print('---------------------')

    for i_z, z in enumerate(Z_filtered):
        total_ones = np.sum(z)
        if total_ones < p:
            if verbose:
                print('z:\n', z)
            # Build X
            idx_x = np.where(z==1)
            idx_x = idx_x[0]
            X = data[:, idx_x]
            X_intercept = np.concatenate((ones_vector, X), axis=1)
            if verbose:
                print('X:\n', X[:3, :])
            idx_y = np.where(z == 0)
            idx_y = idx_y[0]
            Y = data[:, idx_y]
            if verbose:
                print('Y:\n', Y[:3, :])
            beta = np.matmul(np.matmul(np.linalg.inv(np.matmul(X_intercept.T, X_intercept)), X_intercept.T), Y)
            x_from_x_new = x_new[:, idx_x]
            x_from_x_new_intercept = np.concatenate(([[1]], x_from_x_new), axis=1)
            prediction = np.matmul(x_from_x_new_intercept, beta)
            idx = np.concatenate((idx_x, idx_y))
            h = np.concatenate((x_from_x_new, prediction))
            h_sort = np.reshape(h[np.argsort(idx)], newshape=(1, -1))

            if verbose:
                print('beta:\n', beta)
                print('x_from_x_new:\n', x_from_x_new)
                print('x_from_x_new_intercept:\n', x_from_x_new_intercept)
                print('prediction:\n', prediction)
                print('idx_x:\n', idx_x)
                print('idx_y:\n', idx_y)
                print('idx:\n', idx)
                print('h:\n', h)
        else:
            h_sort = np.copy(x_new)

        if verbose:
            print('h_sort:\n', h_sort)
            print('---------------------')
        H[i_z] = h_sort
    if verbose:
        print('Final H:\n', H)
    return H
H = obtain_H(Z, X, x_new, verbose=True)

In [ ]:
# Obtain matrix Y_pred